# McNemar Test

## Import All Necessary Libraries

In [11]:
import pandas as pd
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar 
import os

## Read All the Data

In [12]:
dir = '/Users/angelaoryza/Documents/TA/noisy-rnnids/rnnids-py/results/vector/mcnemar/http'
subdir = os.listdir(dir)

In [13]:
df_dict = {}
df_dict['bi-lstm'] = {}
df_dict['lstm'] = {}
for file in subdir:
    if file.endswith('.csv'):
        df = pd.read_csv(f'./{file}')
        file = file.split('.csv')[0]
        if "bi-lstm" in file:
            file = file.replace('bi-lstm_', '')
            df_dict['bi-lstm'][file] = df
        else:
            file = file.replace('lstm_', '')
            df_dict['lstm'][file] = df

In [14]:
df_dict['bi-lstm']['pred_0.0']

,Unnamed: 0,id,b_mean,b_iqr,b_zscore,f_zscore,f_mean,f_iqr
0,0,175.45.176.1-4657-149.171.126.18-80-tcp,1,1,1,0,0,1
1,1,175.45.176.3-32473-149.171.126.18-80-tcp,0,1,1,0,0,0
2,2,175.45.176.0-21873-149.171.126.14-80-tcp,0,1,1,0,0,0
3,3,175.45.176.0-49194-149.171.126.17-80-tcp,1,1,1,1,0,1
4,4,175.45.176.1-51435-149.171.126.14-80-tcp,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...
154327,154327,149.171.126.15-80-175.45.176.2-20096-tcp,0,1,1,0,0,0
154328,154328,59.166.0.7-25671-149.171.126.9-80-tcp,0,0,0,0,0,0
154329,154329,59.166.0.6-24097-149.171.126.7-80-tcp,0,0,0,0,0,0
154330,154330,59.166.0.6-22144-149.171.126.3-80-tcp,0,0,0,0,0,0


In [15]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        val.drop('Unnamed: 0', axis=1, inplace=True)

## Data Exploration

In [16]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        temp = val[val.duplicated()]['id'].unique()
        print(f'There are {len(temp)} duplicates data in df {k}')

There are 30828 duplicates data in df bi-lstm.001
There are 30546 duplicates data in df pred_0.0
There are 30656 duplicates data in df pred_0.005
There are 30891 duplicates data in df pred_0.004
There are 30608 duplicates data in df pred_0.003
There are 30845 duplicates data in df pred_0.002
There are 26540 duplicates data in df pred_0.001
There are 30764 duplicates data in df pred_0.002
There are 30788 duplicates data in df pred_0.003
There are 30612 duplicates data in df pred_0.001
There are 30821 duplicates data in df pred_0.0
There are 30609 duplicates data in df pred_0.004
There are 30421 duplicates data in df pred_0.005


In [17]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        count = 0
        for t in temp:
            srs = val[val['id']==t][['b_mean', 'b_iqr', 'b_zscore', 'f_mean', 'f_iqr', 'f_zscore' ]].nunique()
            if (srs > 1).any():
                print(t)
                count += 1
                break
            else:
                continue
        if count == 0:
            print(f'All duplicated values in df {key}-{k} have the same prediction')
        else:
            print(f'Not all duplicated values in df {key}-{k} have the same prediction')

149.171.126.3-80-59.166.0.6-3934-tcp
Not all duplicated values in df bi-lstm-bi-lstm.001 have the same prediction
149.171.126.0-80-59.166.0.9-14582-tcp
Not all duplicated values in df bi-lstm-pred_0.0 have the same prediction
149.171.126.0-80-59.166.0.9-14582-tcp
Not all duplicated values in df bi-lstm-pred_0.005 have the same prediction
149.171.126.0-80-59.166.0.9-14582-tcp
Not all duplicated values in df bi-lstm-pred_0.004 have the same prediction
149.171.126.3-80-59.166.0.6-3934-tcp
Not all duplicated values in df bi-lstm-pred_0.003 have the same prediction
149.171.126.3-80-59.166.0.6-3934-tcp
Not all duplicated values in df bi-lstm-pred_0.002 have the same prediction
149.171.126.0-80-59.166.0.9-14582-tcp
Not all duplicated values in df bi-lstm-pred_0.001 have the same prediction
149.171.126.3-80-59.166.0.6-3934-tcp
Not all duplicated values in df lstm-pred_0.002 have the same prediction
149.171.126.0-80-59.166.0.9-14582-tcp
Not all duplicated values in df lstm-pred_0.003 have the s

## Data Pre-Processing

In [18]:
def preprocess(df):
    df = df.sort_values(by='id')
    df.drop_duplicates(inplace=True)
    return df

In [19]:
for key, df in df_dict.items():
    for k, val in df_dict[key].items():
        val = preprocess(val)

### Merge DF

In [20]:
df_merge = dict()
for key, df in df_dict['lstm'].items():
    x = df_dict['bi-lstm'][key]
    df_all = pd.merge(x, df, on='id', how='inner')
    df_all = df_all.sort_values(by='id')
    df_merge[key] = df_all
        

- Plot F1-Score untuk tiap tiap threshold
- Cari trend regresinya, Hitung gradiennya untuk tiap threshold.
- Dilakukan untuk setiap protokol
- Rekap McNemar

- Vectorizing:


## Run McNemar Test

In [21]:
def mcnemar_test(df, x , y, key):
    a = len(df[(df[x]==0) & (df[y]==0)])
    b = len(df[(df[x]==0) & (df[y]==1)])
    c = len(df[(df[x]==1) & (df[y]==0)])
    d = len(df[(df[x]==1) & (df[y]==1)])

    table = np.array([
    [a,b],
    [c,d]
    ])

    print(key)
    print(table)
    result = mcnemar(table)
    print(result)
    return result.pvalue


### For Binary Score

#### For B-Mean

In [22]:
result = {}
result['b-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_mean_x', 'b_mean_y', key)
    result['b-mean'][key] = test

pred_0.002
[[277549   5216]
 [    73   6074]]
pvalue      0.0
statistic   73.0
pred_0.003
[[270351   4458]
 [   134  13969]]
pvalue      0.0
statistic   134.0
pred_0.001
[[270351   1708]
 [  5183  11670]]
pvalue      0.0
statistic   1708.0
pred_0.0
[[271048   2428]
 [   120  15316]]
pvalue      0.0
statistic   120.0
pred_0.004
[[269058    750]
 [   108  18996]]
pvalue      5.297165766641721e-119
statistic   108.0
pred_0.005
[[276934   9152]
 [   249   2577]]
pvalue      0.0
statistic   249.0


#### For B-IQR

In [23]:
result['b-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_iqr_x', 'b_iqr_y', key)
    result['b-iqr'][key] = test

pred_0.002
[[174956   1127]
 [   628 112201]]
pvalue      5.158635818240357e-33
statistic   628.0
pred_0.003
[[180362    635]
 [   509 107406]]
pvalue      0.00021634410866244603
statistic   509.0
pred_0.001
[[168382  85835]
 [  8563  26132]]
pvalue      0.0
statistic   8563.0
pred_0.0
[[173023    158]
 [  7522 108209]]
pvalue      0.0
statistic   158.0
pred_0.004
[[177368   1639]
 [  1151 108754]]
pvalue      2.389972242108363e-20
statistic   1151.0
pred_0.005
[[181272   2719]
 [  1478 103443]]
pvalue      7.339413353434423e-83
statistic   1478.0


#### For B-ZScore

In [24]:
result['b-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'b_zscore_x', 'b_zscore_y', key)
    result['b-zscore'][key] = test

pred_0.002
[[263910   3090]
 [   675  21237]]
pvalue      0.0
statistic   675.0
pred_0.003
[[267514   1017]
 [    24  20357]]
pvalue      2.8134957859710346e-265
statistic   24.0
pred_0.001
[[179671    382]
 [ 87873  20986]]
pvalue      0.0
statistic   382.0
pred_0.0
[[182740    186]
 [ 84918  21068]]
pvalue      0.0
statistic   186.0
pred_0.004
[[183436    493]
 [   333 104650]]
pvalue      2.871971552581072e-08
statistic   333.0
pred_0.005
[[268365   1794]
 [   110  18643]]
pvalue      0.0
statistic   110.0


### For Floating Score

#### For F-Mean

In [25]:
result['f-mean'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_mean_x', 'f_mean_y', key)
    result['f-mean'][key] = test

pred_0.002
[[286084    557]
 [  1385    886]]
pvalue      4.524008747209085e-81
statistic   557.0
pred_0.003
[[270750    153]
 [  4079  13930]]
pvalue      0.0
statistic   153.0
pred_0.001
[[286157   1923]
 [   213    619]]
pvalue      0.0
statistic   213.0
pred_0.0
[[286468    450]
 [   101   1893]]
pvalue      1.5064859183278267e-53
statistic   101.0
pred_0.004
[[268965   6883]
 [    52  13012]]
pvalue      0.0
statistic   52.0
pred_0.005
[[274618   1658]
 [  1652  10984]]
pvalue      0.9307469635181188
statistic   1652.0


#### For F-IQR

In [26]:
result['f-iqr'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_iqr_x', 'f_iqr_y', key)
    result['f-iqr'][key] = test

pred_0.002
[[267603   2804]
 [  2672  15833]]
pvalue      0.07667255575905824
statistic   2672.0


pred_0.003
[[267177    512]
 [   765  20458]]
pvalue      1.4984975661281876e-12
statistic   512.0
pred_0.001
[[256550   4875]
 [ 19741   7746]]
pvalue      0.0
statistic   4875.0
pred_0.0
[[270370    972]
 [ 11123   6447]]
pvalue      0.0
statistic   972.0
pred_0.004
[[266631    662]
 [    83  21536]]
pvalue      6.655482191631931e-113
statistic   83.0
pred_0.005
[[266631   3518]
 [  1044  17719]]
pvalue      9.4888519861126e-310
statistic   1044.0


#### For F-ZScore

In [27]:
result['f-zscore'] = {}
for key, df in df_merge.items():
    test = mcnemar_test(df, 'f_zscore_x', 'f_zscore_y', key)
    result['f-zscore'][key] = test

pred_0.002
[[271009   2213]
 [  3198  12492]]
pvalue      5.004540326918735e-41
statistic   2213.0
pred_0.003
[[267801    875]
 [   407  19829]]
pvalue      9.553807525469777e-40
statistic   407.0


pred_0.001
[[279408   4813]
 [  3069   1622]]
pvalue      1.6385614995626644e-86
statistic   3069.0
pred_0.0
[[276617    643]
 [  6972   4680]]
pvalue      0.0
statistic   643.0
pred_0.004
[[267592    414]
 [    82  20824]]
pvalue      2.2709037218690526e-54
statistic   82.0
pred_0.005
[[268697   4102]
 [   610  15503]]
pvalue      0.0
statistic   610.0


## Saving Test Result

In [28]:
mcnemar_result = pd.DataFrame.from_dict(result).sort_index(axis=0)
mcnemar_result

,b-mean,b-iqr,b-zscore,f-mean,f-iqr,f-zscore
pred_0.0,0.000000e+00,0.000000e+00,0.000000e+00,1.506486e-53,0.000000e+00,0.000000e+00
pred_0.001,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.638561e-86
pred_0.002,0.000000e+00,5.158636e-33,0.000000e+00,4.524009e-81,7.667256e-02,5.004540e-41
pred_0.003,0.000000e+00,2.163441e-04,2.813496e-265,0.000000e+00,1.498498e-12,9.553808e-40
pred_0.004,5.297166e-119,2.389972e-20,2.871972e-08,0.000000e+00,6.655482e-113,2.270904e-54
pred_0.005,0.000000e+00,7.339413e-83,0.000000e+00,9.307470e-01,9.488852e-310,0.000000e+00


In [29]:
mcnemar_result.to_excel('./http-mcnemar.xlsx')